## Question 2 : Adding coordinates for each Postal code

As we need to use dataframe from previous step, same lines of code from 1st question applied

In [20]:
import pandas as pd
import numpy as np

In [21]:
# Loading csv
file_name='https://drive.google.com/uc?authuser=0&id=1gR7wopHIaORTAo6AtkkSHWbeLgdk84l0&export=download'
df=pd.read_csv(file_name)

In [3]:
#dropping rows where Borough = 'Not assigned'
df = df[df.Borough != 'Not assigned']

In [4]:
#assign them value of Borough
df.loc[8,'Neighbourhood'] = "Queen's Park"
df = df.reset_index(drop=True)

In [5]:
#now we need to group df by Borough and concatenate strings form corresponding Neighbourhoods with comma
#df.groupby('Borough')['C'].apply(lambda x: "{%s}" % ', '.join(x))
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda text: ''.join(text.to_string(index=False))).str.replace('(\\n)', ', ').reset_index()

In [6]:
df.shape

(103, 3)

#### Trying to use geocoder  to load coordinates

In [24]:
!conda install -c conda-forge geocoder -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.17.1              |        py36_1000         701 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW packages will be INSTALLED:

    future:   0.17.1-py36_1000 co

In [26]:
import geocoder # import geocoder
coordinates = []
#qcount is a counter for quieries
qcount = 0
#for postal_code in df['Postcode']:
postal_code = df['Postcode'][0]
# initialize your variable to None
lat_lng_coords = None
# loop until you get the coordinates
while (lat_lng_coords is None) and (qcount < 1):
    if (qcount+1)//100 > qcount//100:
        print('{} attempts done'.format(qcount+1))
    qcount = qcount+1
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
#coordinates.append((lat_lng_coords[0],lat_lng_coords[1]))
#print('Got coordinate {}, {} in {} attempts'.format(lat_lng_coords[0],lat_lng_coords[1],qcount))

#### While testing I quickly ran out of 2500 requests limit and got no coordinates in the end. Even with 1K requests no lat_lng_coords was received. Hence had to use simpliest way of using available CSV

In [27]:
# Loading csv of Postal codes and coordinates
file_name='http://cocl.us/Geospatial_data'
postcoordinates = pd.read_csv(file_name)

In [28]:
#quick look at data
postcoordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
#merging dataframes
result = pd.merge(df, postcoordinates, left_on=  ['Postcode'],
                   right_on= ['Postal Code'], 
                   how = 'left')

In [30]:
#as result DataFrame is sorted by postal code, head() would not give same table as in example
#so I have checked couple of postal codes to be sure it agrees with example
result.loc[np.logical_or(result.Postcode == 'M5G',result.Postcode == 'M2H')]

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383
63,M2H,North York,Hillcrest Village,M2H,43.803762,-79.363452
